In [0]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import re
import string
import os

from string import punctuation
from collections import defaultdict
from nltk import FreqDist
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer as countVectorizer
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)


train_df = pd.read_csv("./data/train.csv")
test_df = pd.read_csv("./data/test.csv")
submission = pd.read_csv("./data/sample_submission.csv")

print(train_df.head(10))

data = pd.concat([train_df, test_df], axis=0, sort=False)

print("Number of unique locations: ", data.location.nunique())


print("Missing values:")
data.isna().sum()
data.location.fillna("None", inplace=True)
data['text'] = data['text'].apply(lambda x: remove_URL(x))
data['text'] = data['text'].apply(lambda x: remove_punct(x))

vectorizer = feature_extraction.text.CountVectorizer()

train_v = vectorizer.fit_transform(train_df["text"])
test_v = vectorizer.transform(test_df["text"])

linear_classifier = linear_model.RidgeClassifier()
score = model_selection.cross_val_score(linear_classifier, train_v, train_df["target"], cv=3, scoring="f1")
print(score)
linear_classifier.fit(train_v, train_df["target"])
submission["target"] = linear_classifier.predict(test_v)
print(submission.head())
submission.to_csv('submission.csv', index=False)
